In [203]:
import pandas as pd

df = pd.read_csv('./100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [204]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [205]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [206]:
# vocab
vocab = {'<UNK>':0}

In [207]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [208]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [209]:
len(vocab)

324

In [210]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [211]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [212]:
import torch
from torch.utils.data import Dataset, DataLoader

In [213]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [214]:
dataset = QADataset(df, vocab)

In [215]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [216]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[ 1,  2,  3, 69,  5, 53]]) tensor([260])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([68])
tensor([[ 42, 137, 118,   3, 247,   5, 248]]) tensor([249])
tensor([[  1,   2,   3,   4,   5, 135]]) tensor([136])
tensor([[ 42, 200,   2,  14, 201, 202, 203, 204]]) tensor([205])
tensor([[ 1,  2,  3, 33, 34,  5, 35]]) tensor([36])
tensor([[ 78,  79, 150, 151,  14, 152, 153]]) tensor([154])
tensor([[ 1,  2,  3,  4,  5, 53]]) tensor([54])
tensor([[ 10,  11, 189, 158, 190]]) tensor([191])
tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([246])
tensor([[  1,   2,   3, 122, 123,  19,   3,  45]]) tensor([124])
tensor([[ 10, 140,   3, 141, 171,   5,   3,  70, 172]]) tensor([173])
tensor([[  1,   2,   3, 234,   5, 235]]) tensor([131])
tensor([[ 42, 137,   2,  62,  39,   3, 322, 323]]) tensor([6])
tensor([[ 10, 140,   3, 141, 142,  12, 143,  83,   3, 144]]) tensor([145])
tensor([[ 42,  18, 118,   3, 186, 187]]) tensor([188])
tensor([[ 42, 174,   2,  62,  39, 175, 176,  12, 177, 178]]) tensor([1

In [217]:
import torch.nn as nn

In [218]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [219]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [220]:
learning_rate = 0.001
epochs = 20

In [221]:
model = SimpleRNN(len(vocab))

In [222]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [223]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 529.190436
Epoch: 2, Loss: 459.174004
Epoch: 3, Loss: 378.462847
Epoch: 4, Loss: 317.756575
Epoch: 5, Loss: 265.621948
Epoch: 6, Loss: 217.598685
Epoch: 7, Loss: 173.749781
Epoch: 8, Loss: 135.902210
Epoch: 9, Loss: 104.359691
Epoch: 10, Loss: 80.485982
Epoch: 11, Loss: 62.478967
Epoch: 12, Loss: 49.136205
Epoch: 13, Loss: 39.188592
Epoch: 14, Loss: 31.969162
Epoch: 15, Loss: 26.496164
Epoch: 16, Loss: 22.123362
Epoch: 17, Loss: 18.790733
Epoch: 18, Loss: 15.919029
Epoch: 19, Loss: 13.640504
Epoch: 20, Loss: 11.903867


In [224]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [225]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [226]:
list(vocab.keys())[7]

'paris'